In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "Qwen/Qwen3-0.6B"

# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name, torch_dtype="auto", device_map="auto"
)

`torch_dtype` is deprecated! Use `dtype` instead!


In [8]:
# prepare the model input
prompt = "Describe the content of the r/aww subreddit in 2-3 sentences. Be precise and concise."
messages = [{"role": "user", "content": prompt}]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True,
    enable_thinking=True,  # Switches between thinking and non-thinking modes. Default is True.
)
print(f"Text (Thinking): {text}")
text_no_thinking = tokenizer.apply_chat_template(
    messages, tokenize=False, add_generation_prompt=True, enable_thinking=False
)
print(f"Text (No Thinking): {text_no_thinking}")
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

# conduct text completion
generated_ids = model.generate(**model_inputs, max_new_tokens=32768)
output_ids = generated_ids[0][len(model_inputs.input_ids[0]) :].tolist()

# parsing thinking content
try:
    # rindex finding 151668 (</think>)
    index = len(output_ids) - output_ids[::-1].index(151668)
except ValueError:
    index = 0

thinking_content = tokenizer.decode(output_ids[:index], skip_special_tokens=True).strip(
    "\n"
)
content = tokenizer.decode(output_ids[index:], skip_special_tokens=True).strip("\n")

print("thinking content:", thinking_content)
print("content:", content)

Text (Thinking): <|im_start|>user
Describe the content of the r/aww subreddit in 2-3 sentences. Be precise and concise.<|im_end|>
<|im_start|>assistant

Text (No Thinking): <|im_start|>user
Describe the content of the r/aww subreddit in 2-3 sentences. Be precise and concise.<|im_end|>
<|im_start|>assistant
<think>

</think>


thinking content: <think>
Okay, the user wants me to describe the content of the r/aww subreddit in 2-3 sentences. First, I need to recall what the r/aww subreddit is known for. From what I remember, it's a subreddit focused on gaming, especially the latest in-game content. It's a community where players share their experiences and updates with the game.

Wait, but I should make sure about the specifics. Some sources mention that it's a place for discussing new game updates, in-game events, and community discussions. Also, it's a hub for players to exchange information and get updates. 

I need to be precise and concise. Let me structure it: start with the main fo